<a href="https://colab.research.google.com/github/agolia14/Coupan-Redemption-Prediction/blob/Data-Exploration/Submission_Coupan_Redemption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from collections import Counter

from keras import Sequential
from keras.layers import *

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

from tensorflow.metrics import auc
from keras import regularizers
import tensorflow as tf

from sklearn.utils import shuffle

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [2]:
!git init
!git config --global user.email "agolia14@gmail.com"
!git remote add origin https://d0e5c5d2c5891214a6eb9a978deb96f359421b01@github.com/agolia14/Coupan-Redemption-Prediction.git
!git fetch
!git checkout Data-Exploration

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 195, done.
remote: Total 195 (delta 0), reused 0 (delta 0), pack-reused 195
Receiving objects: 100% (195/195), 11.91 MiB | 12.18 MiB/s, done.
Resolving deltas: 100% (103/103), done.
From https://github.com/agolia14/Coupan-Redemption-Prediction
 * [new branch]      Data-Exploration -> origin/Data-Exploration
 * [new branch]      master           -> origin/master
Branch 'Data-Exploration' set up to track remote branch 'Data-Exploration' from 'origin'.
Switched to a new branch 'Data-Exploration'


In [5]:
raw = pd.read_csv("data/train/train.csv")
campaign_data = np.load("npy/campaign_data.npy", allow_pickle=True)
coup_item_map = pd.read_csv("data/train/coupon_item_mapping.csv")
#cust_trans = np.load("npy/customer_transaction_data.npy", allow_pickle=True)
cust_demographics = np.load("npy/customer_demographics.npy", allow_pickle=True)
item_data = np.load("npy/item_data.npy", allow_pickle=True)
test = pd.read_csv("data/test.csv")
print("Shape = ", raw.shape)
raw.head()

Shape =  (78369, 5)


,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [0]:
raw_campaign = raw.merge(campaign_data, how='left').drop(['campaign_id'],axis=1)
coup_item_data = coup_item_map.merge(item_data, how='left').drop(['item_id'], axis=1).drop_duplicates()
raw_campaign_coupItem = raw_campaign.merge(coup_item_data, how="left")
raw_campaign_coupItem_cust = raw_campaign_coupItem.merge(cust_demographics, how="left")
raw_campaign_coupItem_cust.fillna(-1,inplace=True,downcast='infer')

test_campaign = test.merge(campaign_data, how='left').drop(['campaign_id'],axis=1)
test_campaign_coupItem = test_campaign.merge(coup_item_data, how="left")
test_campaign_coupItem_cust = test_campaign_coupItem.merge(cust_demographics, how="left")
test_campaign_coupItem_cust.fillna(-1,inplace=True,downcast='infer')

In [14]:
raw_campaign_coupItem_cust = shuffle(raw_campaign_coupItem_cust)

X_train = raw_campaign_coupItem_cust.drop(['id','coupon_id','customer_id','campaign_type','start_date','end_date'
                                           ,'redemption_status'], axis=1)
X_test = test_campaign_coupItem_cust.drop(['id','coupon_id','customer_id','campaign_type','start_date','end_date'], axis=1)
y_train = np.array(raw_campaign_coupItem_cust.redemption_status)

print("Shape = ", X_train.shape)
#X.head()

Shape =  (580471, 48)


In [0]:
def create_baseline():
  model = Sequential()
  model.add(Dense(units=60, input_dim=48, kernel_regularizer=regularizers.l2(0.01)))
  Dropout(0.2)
  model.add(BatchNormalization())
  Activation('relu')
  model.add(Dense(units=50))
  Dropout(0.2)
  model.add(BatchNormalization())
  Activation('softmax')
  model.add(Dense(units=25))
  Dropout(0.2)
  model.add(BatchNormalization())
  Activation('relu')
  model.add(Dense(units=1))
  Dropout(0.2)
  model.add(BatchNormalization())
  Activation('softmax')
  model.add(Dense(units=1))
  Dropout(0.2)
  model.add(BatchNormalization())
  Activation('relu')
  model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=[roc_auc, "accuracy"])
  
  return model

In [0]:
def roc_auc(y_true, y_pred):
  return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [11]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=20, batch_size=500, verbose=1)
estimator.fit(X_train, y_train)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
Epoch 1/20
580471/580471 [==============================] - 9s 15us/step - loss: 0.9526 - roc_auc: 0.5263 - acc: 0.8004
Epoch 2/20
580471/580471 [==============================] - 7s 12us/step - loss: 0.2208 - roc_auc: 0.6389 -

In [24]:
pred_nn = estimator.predict(X_test)
pred_train_nn = estimator.predict(X_train)
#pred_nn = pd.DataFrame(pred_nn, columns=['redemption_status'])
idx = pd.DataFrame(test_campaign_coupItem.id)

580471/580471 [==============================] - 2s 4us/step


In [0]:
model = RandomForestClassifier(random_state=10, max_depth=25, n_estimators=20, min_samples_split=20)
model.fit(X_train, y_train)
pred_rfc = model.predict(X_test)
pred_rfc_train = model.predict(X_train)

In [0]:
model = XGBClassifier(random_state=10)
model.fit(X_train, y_train)
pred_xgb = model.predict(X_test)
pred_xgb_train = model.predict(X_train)

In [0]:
pred_nn = pd.DataFrame(pred_nn, columns=['redemption_status'])
pred_train_nn = pd.DataFrame(pred_train_nn, columns=['redemption_status'])
pred_rfc = pd.DataFrame(pred_rfc, columns=['redemption_status'])
pred_rfc_train = pd.DataFrame(pred_rfc_train, columns=['redemption_status'])
pred_xgb = pd.DataFrame(pred_xgb, columns=['redemption_status'])
pred_xgb_train = pd.DataFrame(pred_xgb_train, columns=['redemption_status'])

In [0]:
pred_test = pd.concat([pred_nn,pred_rfc, pred_xgb], axis=1)
pred_train = pd.concat([pred_train_nn ,pred_rfc_train , pred_xgb_train], axis=1)

In [0]:
def repredict():
  model = Sequential()
  model.add(Dense(units=40, input_dim=3, kernel_regularizer=regularizers.l2(0.01)))
  Dropout(0.5)
  model.add(BatchNormalization())
  Activation('relu')
  model.add(Dense(units=20))
  Dropout(0.5)
  model.add(BatchNormalization())
  Activation('softmax')
  model.add(Dense(units=1))
  Dropout(0.5)
  model.add(BatchNormalization())
  Activation('softmax')
  model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=[roc_auc, "accuracy"])
  
  return model

In [56]:
estimator_re = KerasClassifier(build_fn=repredict, epochs=5, batch_size=300, verbose=1)
estimator_re.fit(pred_train, y_train)

Epoch 1/5
580471/580471 [==============================] - 9s 16us/step - loss: 0.4639 - roc_auc: 0.7327 - acc: 0.9488
Epoch 2/5
580471/580471 [==============================] - 7s 12us/step - loss: 0.4499 - roc_auc: 0.7420 - acc: 0.9492
Epoch 3/5
580471/580471 [==============================] - 7s 12us/step - loss: 0.4512 - roc_auc: 0.7425 - acc: 0.9489
Epoch 4/5
580471/580471 [==============================] - 7s 12us/step - loss: 0.4439 - roc_auc: 0.7324 - acc: 0.9488
Epoch 5/5
580471/580471 [==============================] - 7s 12us/step - loss: 0.2466 - roc_auc: 0.6880 - acc: 0.9524


In [58]:
y_pred = estimator_re.predict(pred_test)
y_pred = pd.DataFrame(y_pred, columns=['redemption_status'])
idx = pd.DataFrame(test_campaign_coupItem.id)

y_pred = pd.concat([idx, y_pred], axis=1).drop_duplicates()

y_pred.to_csv("Submission_NN.csv", index=False)

401012/401012 [==============================] - 3s 7us/step
